In [ ]:
# | default_exp feature.enterprise_apps

In [ ]:
# | exporti

from dataclasses import dataclass, field
import datetime as dt

from typing import List, Any
import os

import mbison.client.core as dmda
import mbison.feature.users as dmdu
import mbison.utils as dmut

from nbdev.showdoc import patch_to

In [3]:
# | hide
import nbdev

In [4]:
auth = dmda.DomoAuth(
    domo_instance=os.environ["DOMO_INSTANCE"],
    access_token=os.environ["DOMO_ACCESS_TOKEN"],
)
auth

# design_id = "8c16c8ab-c068-4110-940b-f738d7146efc" # ddx
design_id = '8c16c8ab-c068-4110-940b-f738d7146efc' # enterprise_app


## Routes


In [5]:
# | exports


class App_API_Exception(dmda.API_Exception):
    def __init__(self, res, message=None):
        super().__init__(res=res, message=message)


def get_design_by_id(auth: dmda.DomoAuth, design_id: str, debug_api: bool = False):

    endpoint = f"/api/apps/v1/designs/{design_id}"

    res = dmda.domo_api_request(
        endpoint=endpoint,
        request_type="get",
        auth=auth,
        debug_api=debug_api,
    )

    if not res.is_success:
        raise App_API_Exception(
            res=res, message="is app shared with authenticated user?"
        )

    return res

In [6]:
get_design_by_id(auth=auth, design_id=design_id, debug_api=False).response

{'id': '8c16c8ab-c068-4110-940b-f738d7146efc',
 'name': 'nbdev_documentation',
 'owner': '1893952720',
 'createdBy': '1893952720',
 'createdDate': '2022-12-02T17:26:03Z',
 'updatedBy': '1893952720',
 'updatedDate': '2022-12-02T17:39:38Z',
 'description': None,
 'versions': [],
 'latestVersion': '1.0.0',
 'instances': [],
 'referencingCards': [],
 'owners': [],
 'creator': None,
 'deletedDate': None,
 'trusted': False,
 'hasThumbnail': None}

In [7]:
# | exports


def get_design_versions(auth: dmda.DomoAuth, design_id, debug_api: bool = False):

    endpoint = f"/domoapps/designs/{design_id}/versions"

    return dmda.domo_api_request(
        endpoint=endpoint, auth=auth, request_type="get", debug_api=debug_api
    )

In [8]:
design_versions = get_design_versions(
    auth=auth,
    design_id=design_id,
).response

design_versions

['1.0.0']

In [9]:
# | exports


def get_design_source_code_by_version(
    auth: dmda.DomoAuth,
    download_path,
    design_id,
    version,
    debug_api: bool = False,
):

    download_path = dmut.change_suffix(download_path, ".zip")

    endpoint = f"/domoapps/designs/{design_id}/versions/{version}/assets"

    res = dmda.domo_api_stream_request(
        endpoint=endpoint,
        request_type="get",
        auth=auth,
        debug_api=debug_api,
        download_path=download_path,
    )

    if not res.is_success:
        raise App_API_Exception(
            res=res, message=f"unable to download assets for {design_id}"
        )

    return res

In [10]:
res = get_design_source_code_by_version(
    auth=auth,
    design_id=design_id,
    version=design_versions[0],
    debug_api=False,
    download_path="../../TEST/route.zip",
)

done writing stream


In [11]:
# | exports
def get_all_app_designs(auth: dmda.DomoAuth, debug_api: bool = False):

    endpoint = "/api/apps/v1/designs"

    params = {
        "checkAdminAuthority": True,
        "deleted": False,
        "direction": "desc",
        # "parts" : "owners,creator,thumbnail",
        "search": "",
        "withPermission": "ADMIN",
    }

    res = dmda.domo_api_request(
        endpoint=endpoint,
        request_type="get",
        params=params,
        auth=auth,
        debug_api=debug_api,
    )

    return res

In [12]:
res = get_all_app_designs(auth=auth)
all_apps = res.response

print(len(all_apps))
all_apps[0:1]

75


[{'id': 'e09d6625-3680-4fac-bc71-1cd81607eb9a',
  'name': 'YouTubeViewer Demographic',
  'owner': None,
  'createdBy': None,
  'createdDate': '2016-07-19T22:10:36Z',
  'updatedBy': None,
  'updatedDate': '2016-09-20T21:41:39Z',
  'description': None,
  'versions': [],
  'latestVersion': '1.0.7',
  'instances': [],
  'referencingCards': [],
  'owners': [],
  'creator': None,
  'deletedDate': None,
  'trusted': False,
  'hasThumbnail': None}]

In [25]:
# def get_design_permissions(design_id, auth: dmda.DomoAuth, debug_api: bool = False):
#     endpoint = f"/api/apps/v1/designs/{design_id}/permissions/ADMIN"
#     endpoint = f"/api/apps/v1/designs/dde86741-c198-4b9e-8686-cc60f8dfd5be/permissions/ADMIN"

#     res = dmda.domo_api_request(
#         endpoint=endpoint,
#         request_type="get",
#         auth=auth,
#         debug_api=debug_api,
#     )

#     if not res.is_success:
#         raise App_API_Exception(res=res)

#     return res

In [26]:
# print(design_id)

# get_design_permissions(
#     design_id= design_id,
#     auth = auth,
#     debug_api= False
# ).response

# # next((app for app in all_apps if app["designId"] == design_id), None)

8c16c8ab-c068-4110-940b-f738d7146efc


App_API_Exception:  || 405 - Not allowed || domo-community

In [ ]:
# def share_app(auth : dm):
#     url = 'https://domo-community.domo.com/api/apps/v1/designs/dde86741-c198-4b9e-8686-cc60f8dfd5be/permissions/ADMIN'


## Classes


In [ ]:
# | exports


@dataclass
class DomoEnterpriseApp:
    auth: dmda.DomoAuth = field(repr=False)
    id: str
    name: str
    owner: dmdu.DomoUser
    created_dt: dt.datetime
    lastmodified_dt: dt.datetime
    versions: List[str]
    current_version: str
    referencing_cards: List[dict]

    collection_ddx_source_code  : Any = None

    @classmethod
    def _from_json(cls, obj, auth: dmda.DomoAuth, debug_api: bool = False):

        domo_user = None

        try:
            if obj.get("owner"):
                domo_user = dmdu.DomoUser.get_by_id(
                    user_id=obj["owner"], auth=auth, debug_api=debug_api
                )

        except dmdu.User_API_Exception as e:
            print(e)

        return cls(
            auth=auth,
            id=obj["id"],
            name=obj["name"],
            owner=domo_user,
            created_dt=obj["createdDate"],
            lastmodified_dt=obj["updatedDate"],
            versions=obj["versions"],
            current_version=obj["latestVersion"],
            referencing_cards=obj["referencingCards"],
        )

    @classmethod
    def get_by_id(
        cls,
        design_id,
        auth: dmda.DomoAuth,
        debug_api: bool = False,
        return_raw: bool = False,
    ):
        res = get_app_by_id(auth=auth, design_id=design_id, debug_api=debug_api)

        if return_raw:
            return res

        return cls._from_json(obj=res.response, auth=auth, debug_api=debug_api)

    def _get_source_code_enterprise_app(
        self,
        version: str = None,
        debug_api: bool = False,
        download_folder="./EXPORT/",
        file_name=None,
    ):

        file_name = file_name or f"{self.id} - {version or self.current_version}.zip"
        file_name = dmut.change_suffix(file_name, ".zip")

        download_path = os.path.join(download_folder, file_name)

        return get_app_source_by_version(
            auth=self.auth,
            download_path=download_path,
            design_id=self.id,
            version=version or self.current_version,
            debug_api=debug_api,
        )

    def get_versions(self, debug_api: bool = False, return_raw: bool = False):

        res = get_app_versions(auth=self.auth, design_id=self.id, debug_api=debug_api)
        if return_raw:
            return res

        self.versions = res.response
        return self.versions

In [ ]:

domo_app = DomoEnterpriseApp.get_by_id(design_id=design_id, auth=auth)

domo_app.get_versions()
domo_app.current_version = domo_app.versions[-1]
domo_app

In [ ]:
@patch_to(DomoEnterpriseApp)
def _get_source_code_collection_for_ddx(self, card_id, auth, debug_api: bool = False):
    """
    identifies the collection that stores the source code for a ddx brick

    note card metadata includes "domoapps" which identifies the datastore that feeds an app.
    the one or many collections can be associated with a datastore / app
    for ddx, the source code is stored in the collection 'ddx_app_client_code'
    """

    import mbison.feature.cards as dmac
    import mbison.feature.appdb as dmdb

    card_meta = dmac.get_card_by_id(
        card_id=card_id, auth=auth, debug_api=debug_api
    ).response

    datastore_id = card_meta["domoapp"]["id"]

    collections = dmdb.get_collections(
        auth=auth, debug_api=debug_api, datastore_id=datastore_id
    ).response

    source_code_collection = next(
        (
            collection
            for collection in collections
            if collection["name"] == "ddx_app_client_code"
        ),
        None,
    )

    if not source_code_collection:
        raise Exception(f"unable to to retrieve source code collection for {card_id}")
    
    self.collection_ddx_source_code = dmdb.AppDbCollection.get_by_id(auth = auth, collection_id = source_code_collection['id'], debug_api = debug_api)

    return self.collection_ddx_source_code

In [ ]:
domo_app.get_source_code(download_folder="../../TEST")

In [ ]:
# | export


@dataclass
class DomoEnterpriseApps:
    auth: dmda.DomoAuth = field(repr=False)
    enterprise_apps: List[DomoEnterpriseApp] = None

    def get_apps(self, debug_api: bool = False, return_raw: bool = False):
        res = get_all_apps(auth=self.auth, debug_api=debug_api)

        if return_raw:
            return res

        self.enterprise_apps = [
            DomoEnterpriseApp._from_json(auth=self.auth, obj=obj)
            for obj in res.response
        ]
        return self.enterprise_apps

In [ ]:
DomoEnterpriseApps(auth=auth).get_apps(debug_api=False, return_raw=False)[0:5]

In [ ]:
# | hide

nbdev.nbdev_export("./enterprise_apps.ipynb")